# Libraries

In [ ]:
import os
from pathlib import Path
import numpy as np
from pycocotools.coco import COCO
from PIL import Image
import PIL.Image
from collections import Counter
from fastai.vision.all import *

# Paths for annotations and output

In [20]:
BASE_DIR = os.getcwd()
ANNOTATIONS_PATH = os.path.join(BASE_DIR, 'annotations', '_annotations.coco.json')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Color map for each category from RoboFlow

In [21]:
COLOR_MAP = {
                'agua': (61, 61, 245),
                'erosao': (221, 255, 51),
                'trinca': (252, 128, 7),
                'ruptura': (36, 179, 83)
            }

# COCO file initialization and category mapping

In [22]:
coco = COCO(ANNOTATIONS_PATH)
categories = coco.loadCats(coco.getCatIds())
id_to_name = {cat['id']: cat['name'].lower() for cat in categories}
print("\nDetected categories:")
for k, v in id_to_name.items():
    print(f"  {k}: {v}")

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!

Detected categories:
  0: erosao-gwwa-dvqo
  1: agua
  2: erosao
  3: ruptura
  4: trinca


# Generate masks for each image

In [23]:
for img_id in coco.imgs:
    img_info = coco.imgs[img_id]
    print(f"\nGenerating mask for: {img_info['file_name']}")
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    height, width = img_info['height'], img_info['width']
    colored_mask = np.zeros((height, width, 3), dtype=np.uint8)
    for ann in anns:
        if ann.get('iscrowd', 0) == 1:
            continue
        if not ann.get('segmentation'):
            continue
        cat_name = id_to_name.get(ann['category_id'])
        if cat_name not in COLOR_MAP:
            continue
        ann_mask = coco.annToMask(ann)
        if ann_mask.sum() == 0:
            continue
        colored_mask[ann_mask == 1] = COLOR_MAP[cat_name]
    mask_name = os.path.splitext(img_info['file_name'])[0] + '.png'
    output_path = os.path.join(OUTPUT_DIR, mask_name)
    Image.fromarray(colored_mask).save(output_path)
    print(f"✔ mask saved at: {output_path}")
print("\n🎉 Processing completed! All masks have been generated.")


Generating mask for: erosao_26_JPG.rf.a97b3c357d38b555a576e9f9d55f9ff2.jpg
✔ mask saved at: c:\git-projetos\geoeyeai\output\erosao_26_JPG.rf.a97b3c357d38b555a576e9f9d55f9ff2.png

Generating mask for: agua_10_JPG.rf.97958a8cc523273696b74f8f95c3ce38.jpg
✔ mask saved at: c:\git-projetos\geoeyeai\output\agua_10_JPG.rf.97958a8cc523273696b74f8f95c3ce38.png

Generating mask for: agua_40_JPG.rf.e1578967546da44f2a7a0f77b46885f2.jpg
✔ mask saved at: c:\git-projetos\geoeyeai\output\agua_40_JPG.rf.e1578967546da44f2a7a0f77b46885f2.png

Generating mask for: ruptura_7_jpg.rf.fac4fa0b92e6fc9d210bd6e68591f5ff.jpg
✔ mask saved at: c:\git-projetos\geoeyeai\output\ruptura_7_jpg.rf.fac4fa0b92e6fc9d210bd6e68591f5ff.png

Generating mask for: erosao_412_jpg.rf.0f2ff3a6690d7e68fb9f130aa819df6d.jpg
✔ mask saved at: c:\git-projetos\geoeyeai\output\erosao_412_jpg.rf.0f2ff3a6690d7e68fb9f130aa819df6d.png

Generating mask for: erosao_99_JPG.rf.fce4407d8fe4a308cbcc18c08f464d10.jpg
✔ mask saved at: c:\git-projetos\ge

# Count unique colors in the generated masks

In [24]:
path_masks = OUTPUT_DIR
arquivos = get_image_files(path_masks)
print(f"📂 Reading {len(arquivos)} masks... please wait.")

# Contadores para armazenar as cores encontradas
cores_encontradas = Counter()
tipos_arquivos = Counter()

for arquivo in arquivos:
    # Abre a imagem
    img = PIL.Image.open(arquivo)
    arr = np.array(img)
    
    # Verifica se é Grayscale (2D) ou RGB (3D)
    if len(arr.shape) == 2:
        tipos_arquivos['Grayscale (2D)'] += 1
        # Pega valores únicos
        uniques = np.unique(arr)
        cores_encontradas.update(uniques)
        
    elif len(arr.shape) == 3:
        tipos_arquivos['RGB (3D)'] += 1
        # Transforma a matriz 3D em uma lista de pixels (R, G, B)
        # Ex: transforma (1024, 1024, 3) em (1048576, 3)
        pixels = arr.reshape(-1, 3)
        # Pega as linhas únicas (cores únicas)
        uniques = np.unique(pixels, axis=0)
        # Adiciona ao contador (convertendo para tupla para poder contar)
        cores_encontradas.update([tuple(p) for p in uniques])

print("-" * 40)
print("📊 Statistics")
print("-" * 40)
print(f"File types found: {dict(tipos_arquivos)}")
print("\n🎨 THE 10 MOST COMMON COLORS (Format: Color -> How many images have this color):")
for cor, contagem in cores_encontradas.most_common():
    print(f"   👉 Color: {cor} \t(Appears in pixels of various images)")
print("-" * 40)

📂 Reading 1533 masks... please wait.


KeyboardInterrupt: 

# Build a pixel map for Fast AI

In [ ]:
PATH_MASKS = OUTPUT_DIR
NOMES_CONHECIDOS = {
                        (0, 0, 0): "Background",
                        (61, 61, 245): "Agua",
                        (221, 255, 51): "Erosao",
                    }

In [ ]:
def analisar_dataset(path_masks):
    arquivos = list(path_masks.glob("*.png")) + list(path_masks.glob("*.jpg")) # Ajuste extensões se precisar
    print(f"📂 Lendo {len(arquivos)} máscaras...")

    todas_cores = Counter()

    for arquivo in arquivos:
        img = PIL.Image.open(arquivo)
        arr = np.array(img)

        # Se for 3D (RGB)
        if len(arr.shape) == 3:
            pixels = arr.reshape(-1, 3)
            # Pega cores únicas dessa imagem
            cores_img = np.unique(pixels, axis=0)
            # Adiciona ao contador geral (transforma em tupla para poder contar)
            todas_cores.update([tuple(c) for c in cores_img])
        
        # Se for 2D (Grayscale/Indexed)
        elif len(arr.shape) == 2:
            cores_img = np.unique(arr)
            todas_cores.update(cores_img)

    return todas_cores

# --- EXECUÇÃO ---
cores_encontradas = analisar_dataset(PATH_MASKS)

# Filtra ruído (cores que aparecem em menos de 1000 pixels no total do dataset)
# Isso evita que um pixelzinho borrado crie uma classe nova errada
LIMITE_RUIDO = 1000 
cores_validas = [c for c, qtd in cores_encontradas.items() if qtd > LIMITE_RUIDO]

# --- ORDENAÇÃO INTELIGENTE ---
# 1. Background sempre primeiro (0,0,0) ou 0
# 2. Depois, ordena por frequência (quem tem mais pixels ganha ID menor)
#    OU ordena fixo se você quiser garantir consistência sempre.

# Vamos separar o background
tem_bg_rgb = (0,0,0) in cores_validas
tem_bg_gray = 0 in cores_validas

lista_final = []

# Adiciona Background primeiro
if tem_bg_rgb:
    lista_final.append((0,0,0))
    cores_validas.remove((0,0,0))
elif tem_bg_gray:
    lista_final.append(0)
    cores_validas.remove(0)

# Ordena o resto pela quantidade de pixels (do mais comum para o menos comum)
# Isso explica pq Erosão pode estar vindo antes da Água: ela pode ter mais pixels no total!
resto_ordenado = sorted(cores_validas, key=lambda x: cores_encontradas[x], reverse=True)
lista_final.extend(resto_ordenado)

# --- GERAR O CODIGO ---
print("\n" + "="*40)
print("✅ AQUI ESTÁ SEU MAPEAMENTO FINAL")
print("="*40)

pixel_map = {}
meus_codes = []

for idx, cor in enumerate(lista_final):
    # Tenta achar o nome
    nome = NOMES_CONHECIDOS.get(cor, f"Classe_{idx}")
    
    pixel_map[cor] = idx
    meus_codes.append(nome)
    
    print(f"ID {idx}: {str(cor):<15} -> {nome}")

print("\nCOPIE E COLE ISSO NO SEU CODIGO DE TREINO:")
print("-" * 30)
print(f"pixel_map = {pixel_map}")
print(f"meus_codes = {meus_codes}")
print("-" * 30)

if len(cores_encontradas) > len(lista_final):
    print(f"\n⚠️  Atenção: {len(cores_encontradas) - len(lista_final)} cores raras foram ignoradas como ruído.")